In [ ]:
#To generate the lookup table
def generateTable(data,k):   
    T = {}
    #print(len(data))
    for k in range(1,6):
      for i in range(len(data)-k):
          X = data[i:i+k]
          Y = data[i+k]
          if T.get(X) is None:
              T[X] = {}
              T[X][Y] = 1
          else:
              if T[X].get(Y) is None:
                  T[X][Y] = 1
              else:
                  T[X][Y] += 1
          #print(X,Y)
    return T

In [ ]:
""" {
  merg:{
    e:2/9,
    m:3/9,
    ' ':4/9,
  }
  freq:{
    e:,
    g:,
  }
}
"""

" {\n  merg:{\n    e:2/9,\n    m:3/9,\n    ' ':4/9,\n  }\n  freq:{\n    e:,\n    g:,\n  }\n}\n"

In [ ]:
def freqIntoProb(T):     
    for kx in T.keys():
        s = float(sum(T[kx].values()))
        for k in T[kx].keys():
            T[kx][k] = T[kx][k]/s
                
    return T

In [ ]:
datasetPath = "merged_hindi_corpus.txt"

def loadText(filename):
    with open(filename,encoding='utf8') as f:
        return f.read().lower()
    
text = loadText(datasetPath)
print("Dataset Loaded")

Dataset Loaded


In [ ]:
def MarkovChain(text,k):
    T = generateTable(text,k)
    T = freqIntoProb(T)
    return T
 
model = MarkovChain(text)
print('Model Created.')

Model Created.


In [ ]:
import numpy as np

def sample_next(ctx,model):
    possi_char = list()
    possi_val = list()
    sum=0
    for i in range(1,len(ctx)+1):
      ch = ctx[-i:]
      if model.get(ch) is not None:
        #print(model[ch])
        poss_ch = list(model[ch].keys())
        poss_val = list(model[ch].values())
        idx = np.argmax(poss_val)
        sum+= i*poss_val[idx]
        possi_char.append(poss_ch[idx])
        possi_val.append(i*poss_val[idx])
    
    for j in range(len(possi_val)):
      possi_val[j]/=sum
    
    #return possi_char[np.argmax(possi_val)]

    #ctx = ctx[-k:]
    #if model.get(ctx) is None:
    #    return " "
    possible_Chars = list(model[ctx].keys())
    possible_values = list(model[ctx].values())
    #print(possible_Chars)
    #print(possible_values)
    #return possible_Chars[np.argmax(possible_values)]
    return np.random.choice(possible_Chars,p=possible_values)

In [ ]:
sample_next("भा",model)

'ज'

In [ ]:
"""#input
word=input("Enter word  :  ")
while(word != ''):
  word+=sample_next(word,model)
  if(sample_next(word,model)==' '):
    break
  else:
    word+=sample_next(word,model)

print(word)"""

In [ ]:
def generateText(starting_sent,k,maxSize=1000):
    
    sentence = starting_sent
    ctx = starting_sent[-k:]
    
    for ix in range(maxSize):
        next_prediction = sample_next(ctx,model)
        sentence += next_prediction
        ctx = sentence[-k:]
    return sentence

inpText=input("Enter a word or text : ") 
text = generateText(inpText, k, maxSize=300)
print(text)

In [ ]:
accuracy=[]
k_values=[]
text=load_text(datasetPath)
text=text.split(" ")
for k in range(0,40):
  model=MarkovChain(text,k)
  correct=0
  for i in range(len(text)-k):
    Xtest=text[i:i+k]
    Ytest=text[i+k]
    Ypred=generateText(Xtest,model,k)
    if(Ytest==Ypred):
      correct+=1
  accuracy.append(100 * (correct/len(text)))
  k_values.append(k)
 # print("Accuracy : {}, k : {}".format(correct/len(text),k))

import matplotlib.pyplot as plt
plt.plot(k_values,accuracy)
plt.axhline(y=100, c='orange')
plt.xlabel("n (n-grams)")
plt.ylim(0,115)
plt.title("Accuracy for character generation using Markov Vs n-grams")
plt.ylabel("Accuracy (%)")
plt.show()